In [1]:
from auto_login import get_curr_path,get_credentials,auto_login,generate_access_token
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import ta

In [2]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)


In [3]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [14]:
instrument_df.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


In [15]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [16]:
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [22]:
infy_df = fetchOHLC('INFY','5minute',5)
infy_df.to_csv('infy.csv')

In [23]:
infy_df.head()

,open,high,low,close,volume
date,,,,,
2024-05-31 09:15:00+05:30,1409.80,1414.15,1405.10,1413.30,640646
2024-05-31 09:20:00+05:30,1414.00,1414.75,1409.70,1411.20,564481
2024-05-31 09:25:00+05:30,1411.25,1415.45,1410.40,1413.45,484983
2024-05-31 09:30:00+05:30,1412.95,1414.70,1412.60,1414.30,610608
2024-05-31 09:35:00+05:30,1414.30,1414.30,1407.65,1409.80,239481


In [24]:
df = infy_df.copy()

In [25]:
def compute_bollinger_bands(df,n):
    df['sma'] = df['close'].rolling(window=20).mean()
    df['upper_band'] = df['sma'] + (2*df['close'].rolling(window=20).std(ddof=0))
    df['lower_band'] = df['sma'] - (2*df['close'].rolling(window=20).std(ddof=0))
    df['band_width'] = df['upper_band'] - df['lower_band']
    return df

In [26]:
df = compute_bollinger_bands(df,20)
df

,open,high,low,close,volume,sma,upper_band,lower_band,band_width
date,,,,,,,,,
2024-05-31 09:15:00+05:30,1409.80,1414.15,1405.10,1413.30,640646,NaN,NaN,NaN,NaN
2024-05-31 09:20:00+05:30,1414.00,1414.75,1409.70,1411.20,564481,NaN,NaN,NaN,NaN
2024-05-31 09:25:00+05:30,1411.25,1415.45,1410.40,1413.45,484983,NaN,NaN,NaN,NaN
2024-05-31 09:30:00+05:30,1412.95,1414.70,1412.60,1414.30,610608,NaN,NaN,NaN,NaN
2024-05-31 09:35:00+05:30,1414.30,1414.30,1407.65,1409.80,239481,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2024-06-05 14:15:00+05:30,1426.30,1429.35,1425.05,1428.00,138590,1429.1900,1433.407535,1424.972465,8.435070
2024-06-05 14:20:00+05:30,1427.45,1430.95,1426.30,1430.50,93242,1429.1650,1433.345084,1424.984916,8.360167
2024-06-05 14:25:00+05:30,1430.50,1431.30,1428.40,1429.30,76548,1429.1700,1433.350478,1424.989522,8.360957


In [27]:
df.to_csv('bollinger.csv')